In [2]:
import pymc as pm
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming data loading and preprocessing have been done as previously discussed
df = pd.read_csv('diabetes.csv')
# Encoding categorical variables using LabelEncoder
le = LabelEncoder()
categorical_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
for col in categorical_columns:
    data[col] = le.fit_transform(data[col])

# Preparing the dataset for regression (Example: Predict 'bmi' based on other features excluding 'id' and 'stroke')
X = data.drop(columns=['id', 'bmi', 'stroke']).values
y = data['bmi'].values.reshape(-1, 1)

n, p = X.shape

with pm.Model() as MLR:
    # Priors for regression coefficients
    betas = pm.MvNormal('betas', mu=np.zeros(p), cov=np.eye(p), shape=p)

    # Experiment with different priors for sigma
    # Option 1: Half-Normal
    sigma_half_normal = pm.HalfNormal('sigma_half_normal', sigma=5)

    # Option 2: Inverse-Gamma
    alpha, beta = 3, 2  # Example hyperparameters
    sigma_inverse_gamma = pm.InverseGamma('sigma_inverse_gamma', alpha=alpha, beta=beta)

    # Option 3: Exponential
    lambda_rate = 1
    sigma_exponential = pm.Exponential('sigma_exponential', lam=lambda_rate)

    # Option 4: Half-Cauchy
    beta_cauchy = 5
    sigma_half_cauchy = pm.HalfCauchy('sigma_half_cauchy', beta=beta_cauchy)

    # Choose one of the sigma priors for the model (comment out others as needed)
    sigma = sigma_half_normal  # Change this line to use different priors

    # Linear model for the mean of the observed data
    mu = pm.math.dot(X, betas)

    # Likelihood of the observed data
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y.ravel())

    # Sampling from the posterior
    idata = pm.sample(return_inferencedata=True, target_accept=0.95, chains=2, draws=1000)

# After sampling, to view the summary of the posterior distributions for the parameters
summary = pm.summary(idata)
print(summary)

# You might also generate posterior predictive checks
with MLR:
    ppc = pm.sample_posterior_predictive(idata)



NameError: name 'data' is not defined

Given the posterior distribution
\begin{align*}
\beta|\Sigma, X, y
\end{align*}
expressed in terms of the covariance matrix
\begin{align*}
\Sigma,
\end{align*}
we can re-express it in terms of the variance
\begin{align*}
\sigma^2
\end{align*}
by substituting  
\begin{align*}
\Sigma  
\end{align*}  
with
\begin{align*}
\sigma^2I
\end{align*}
where  
\begin{align*}
\sigma^2 = \frac{1}{\tau}.
\end{align*}
This substitution reflects a simplifying assumption that the error variances are identical across all dimensions and independent, leading to a diagonal covariance matrix. Consequently, the inverse of the covariance matrix,
\begin{align*}
\Sigma^{-1},
\end{align*}
becomes
\begin{align*}
\tau I,
\end{align*}
where
\begin{align*}
I
\end{align*}
is the identity matrix and
\begin{align*}
\tau
\end{align*}
is the precision, the reciprocal of the variance. The updated expressions for the expectation and variance of the posterior distribution of
\begin{align*}
\beta
\end{align*}
are:

\begin{align*}
\mathbb{E}[\beta] &= (X^TX + \tau I)^{-1}X^Ty \\
\text{Var}[\beta] &= (X^TX + \tau I)^{-1}
\end{align*}

These equations represent the mean and variance of the posterior distribution for
\begin{align*}
\beta
\end{align*}
in a Bayesian linear regression model, where the prior distribution of
\begin{align*}
\beta
\end{align*}
is assumed to be multivariate normal with mean zero and variance
\begin{align*}
\sigma^2I.
\end{align*}


\begin{align*}
&\text{The expected value of } \beta \text{ given the covariance matrix } \Sigma, \text{ the design matrix } X, \text{ and the observed outcomes } y, \text{ denoted as } \mathbb{E}[\beta|\Sigma, X, y], \text{ is calculated using the formula:} \\
&\mathbb{E}[\beta|\Sigma, X, y] = (X^TX + \Sigma^{-1})^{-1}X^Ty
\end{align*}
This represents the posterior mean of the regression coefficients in a Bayesian multivariate normal regression model, where $\Sigma$ is the prior covariance matrix of the coefficients, $X$ is the matrix of predictors, and $y$ is the vector of observed responses. The term $(X^TX + \Sigma^{-1})^{-1}$ is the precision matrix of the posterior distribution, and $X^Ty$ is the product of the transpose of $X$ and $y$.





To make the expectation $\mathbb{E}[\beta|\Sigma, X, y]$ equal to $(X^TX)^{-1}X^Ty$, we would need $\Sigma^{-1}$ to be the zero matrix. However, this is not possible because $\Sigma$ is a covariance matrix and must be positive semi-definite, thus its inverse (if it exists) is positive definite and cannot be the zero matrix.

The only scenario where $\mathbb{E}[\beta|\Sigma, X, y]$ would equal $(X^TX)^{-1}X^Ty$ is in the limit as the prior variance goes to infinity, which reflects a non-informative prior for $\beta$. This can be represented mathematically as:

\begin{align*}
\lim_{\Sigma \to \infty} \mathbb{E}[\beta|\Sigma, X, y] &= \lim_{\Sigma \to \infty} (X^TX + \Sigma^{-1})^{-1}X^Ty \\
&= (X^TX)^{-1}X^Ty
\end{align*}

In practice, setting a very large value for the elements of $\Sigma$ (or a very small value for elements of $\Sigma^{-1}$) approximates this condition, but no finite value will exactly yield the ordinary least squares (OLS) estimator.


For the expectation $\mathbb{E}[\beta|\Sigma, X, y]$ to equal $(X^TX)^{-1}X^Ty$, we require that $\Sigma^{-1}$, the precision matrix of the prior distribution, approaches the zero matrix. However, since $\Sigma$ is a covariance matrix, it is by definition positive semi-definite, and its inverse is positive definite, thus it cannot be the zero matrix. The scenario where the expectation equals the ordinary least squares (OLS) estimator is in the limit as the prior variance goes to infinity. Mathematically, this is expressed as:

\begin{align*}
\lim_{\Sigma \to \infty} \mathbb{E}[\beta|\Sigma, X, y] &= \lim_{\Sigma \to \infty} (X^TX + \Sigma^{-1})^{-1}X^Ty \\
&= (X^TX)^{-1}X^Ty
\end{align*}

This implies a non-informative prior for $\beta$, where the prior variance is infinite and the prior's influence on the posterior is negligible.



\begin{align*}\text{Var}[\beta|\Sigma, X, y] = (X^TX + \Sigma^{-1})^{-1}\end{align*}


In [1]:
import pymc as pm
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the dataset (make sure to adjust the path to where the dataset is located on your machine)
df = pd.read_csv('diabetes.csv')

# Drop 'id' as it is not a feature and fill missing 'bmi' values with the median
data.drop(columns='id', inplace=True)
data['bmi'].fillna(data['bmi'].median(), inplace=True)

# Encode categorical features
categorical_features = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
for feature in categorical_features:
    data[feature] = LabelEncoder().fit_transform(data[feature])

# Define the feature matrix X and target vector y
X = data.drop(columns='stroke').values  # assuming we want to predict the likelihood of stroke
y = data['stroke'].values

# Define the number of features
p = X.shape[1]

# Bayesian Multivariate Normal Model using PyMC
with pm.Model() as MNV_LKJ:
    # Define the packed L matrix for the Cholesky decomposition
    packed_L = pm.LKJCholeskyCov('packed_L', n=p, eta=2.0, sd_dist=pm.Exponential.dist(1.0, shape=p), compute_corr=False)
    # Unpack L to lower triangular matrix L
    L = pm.expand_packed_triangular(p, packed_L)
    # Define the mean vector for the multivariate normal distribution
    mu = pm.Normal('mu', mu=np.zeros(p), sigma=1.0, shape=p)
    # Define the observed data likelihood
    y_obs = pm.MvNormal('y_obs', mu=mu, chol=L, observed=y)

    # Perform sampling
    trace = pm.sample(1000, return_inferencedata=True)

# You can inspect the trace using trace.to_dataframe() in your local environment


FileNotFoundError: [Errno 2] No such file or directory: 'healthcare-dataset-stroke-data.csv'